# RSNA2024 LSDC Submission Baseline

This code outlines a process for classifying degenerative lumbar spine conditions using MRI images. It starts by importing necessary libraries for data handling, model training, and image processing, including PyTorch for deep learning and Albumentations for data augmentation.

A configuration section sets parameters such as device type (CPU/GPU), image size, and model specifications (EfficientNet). The dataset is then defined using a custom RSNA24TestDataset class, which loads and processes DICOM images (MRI scans) into a standardized format suitable for model input.

The code then defines a neural network model based on a pre-trained EfficientNet architecture and loads several pre-trained model weights for inference. An inference loop processes the test dataset, passing images through the ensemble of models and aggregating the predictions.

Finally, the predictions are formatted into a submission file, which is saved for evaluation in the competition. The notebook concludes by acknowledging potential areas for improvement in the approach used.


Training Baseline V18

# Import Libralies

In [1]:
import os
import gc
import sys
from PIL import Image
import cv2
import math, random
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

from collections import OrderedDict

import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW

import timm
from timm.utils import ModelEmaV2
from transformers import get_cosine_schedule_with_warmup

import albumentations as A

from sklearn.model_selection import KFold

import re
import pydicom

In [2]:
rd = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification'


# Config

In [3]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
N_WORKERS = os.cpu_count()
USE_AMP = False # True if we can use GPU. 
SEED = 42

IMG_SIZE = [512, 512]
IN_CHANS = 30
N_LABELS = 25
N_CLASSES = 3 * N_LABELS

N_FOLDS = 2

# MODEL_NAME = "tf_efficientnet_b3.ns_jft_in1k"
MODEL_NAME = "tf_efficientnet_b5.ns_jft_in1k"



BATCH_SIZE = 1

In [4]:
rd = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification'

In [5]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cpu')

In [6]:
df = pd.read_csv(f'{rd}/test_series_descriptions.csv')
df.head()

,study_id,series_id,series_description
0,44036939,2828203845,Sagittal T1
1,44036939,3481971518,Axial T2
2,44036939,3844393089,Sagittal T2/STIR


In [7]:
study_ids = list(df['study_id'].unique())

In [8]:
sample_sub = pd.read_csv(f'{rd}/sample_submission.csv')

In [9]:
LABELS = list(sample_sub.columns[1:])
LABELS

['normal_mild', 'moderate', 'severe']

In [10]:
CONDITIONS = [
    'spinal_canal_stenosis', 
    'left_neural_foraminal_narrowing', 
    'right_neural_foraminal_narrowing',
    'left_subarticular_stenosis',
    'right_subarticular_stenosis'
]

LEVELS = [
    'l1_l2',
    'l2_l3',
    'l3_l4',
    'l4_l5',
    'l5_s1',
]

In [11]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

# Define Dataset

In [12]:
class RSNA24TestDataset(Dataset):
    def __init__(self, df, study_ids, phase='test', transform=None):
        self.df = df
        self.study_ids = study_ids
        self.transform = transform
        self.phase = phase
    
    def __len__(self):
        return len(self.study_ids)
    
    def get_img_paths(self, study_id, series_desc):
        pdf = self.df[self.df['study_id']==study_id]
        pdf_ = pdf[pdf['series_description']==series_desc]
        allimgs = []
        for i, row in pdf_.iterrows():
            pimgs = glob.glob(f'{rd}/test_images/{study_id}/{row["series_id"]}/*.dcm')
            pimgs = sorted(pimgs, key=natural_keys)
            allimgs.extend(pimgs)
            
        return allimgs
    
    def read_dcm_ret_arr(self, src_path):
        dicom_data = pydicom.dcmread(src_path)
        image = dicom_data.pixel_array
        image = (image - image.min()) / (image.max() - image.min() + 1e-6) * 255
        img = cv2.resize(image, (IMG_SIZE[0], IMG_SIZE[1]),interpolation=cv2.INTER_CUBIC)
        assert img.shape==(IMG_SIZE[0], IMG_SIZE[1])
        return img

    def __getitem__(self, idx):
        x = np.zeros((IMG_SIZE[0], IMG_SIZE[1], IN_CHANS), dtype=np.uint8)
        st_id = self.study_ids[idx]        
        
        # Sagittal T1
        allimgs_st1 = self.get_img_paths(st_id, 'Sagittal T1')
        if len(allimgs_st1)==0:
            print(st_id, ': Sagittal T1, has no images')
        
        else:
            step = len(allimgs_st1) / 10.0
            st = len(allimgs_st1)/2.0 - 4.0*step
            end = len(allimgs_st1)+0.0001
            for j, i in enumerate(np.arange(st, end, step)):
                try:
                    ind2 = max(0, int((i-0.5001).round()))
                    img = self.read_dcm_ret_arr(allimgs_st1[ind2])
                    x[..., j] = img.astype(np.uint8)
                except:
                    print(f'failed to load on {st_id}, Sagittal T1')
                    pass
            
        # Sagittal T2/STIR
        allimgs_st2 = self.get_img_paths(st_id, 'Sagittal T2/STIR')
        if len(allimgs_st2)==0:
            print(st_id, ': Sagittal T2/STIR, has no images')
            
        else:
            step = len(allimgs_st2) / 10.0
            st = len(allimgs_st2)/2.0 - 4.0*step
            end = len(allimgs_st2)+0.0001
            for j, i in enumerate(np.arange(st, end, step)):
                try:
                    ind2 = max(0, int((i-0.5001).round()))
                    img = self.read_dcm_ret_arr(allimgs_st2[ind2])
                    x[..., j+10] = img.astype(np.uint8)
                except:
                    print(f'failed to load on {st_id}, Sagittal T2/STIR')
                    pass
            
        # Axial T2
        allimgs_at2 = self.get_img_paths(st_id, 'Axial T2')
        if len(allimgs_at2)==0:
            print(st_id, ': Axial T2, has no images')
            
        else:
            step = len(allimgs_at2) / 10.0
            st = len(allimgs_at2)/2.0 - 4.0*step
            end = len(allimgs_at2)+0.0001

            for j, i in enumerate(np.arange(st, end, step)):
                try:
                    ind2 = max(0, int((i-0.5001).round()))
                    img = self.read_dcm_ret_arr(allimgs_at2[ind2])
                    x[..., j+20] = img.astype(np.uint8)
                except:
                    print(f'failed to load on {st_id}, Axial T2')
                    pass  
            
            
        if self.transform is not None:
            x = self.transform(image=x)['image']

        x = x.transpose(2, 0, 1)
                
        return x, str(st_id)

In [13]:
transforms_test = A.Compose([
    A.Resize(IMG_SIZE[0], IMG_SIZE[1]),
    A.Normalize(mean=0.5, std=0.5)
])

In [14]:
test_ds = RSNA24TestDataset(df, study_ids, transform=transforms_test)
test_dl = DataLoader(
    test_ds, 
    batch_size=1, 
    shuffle=False,
    num_workers=N_WORKERS,
    pin_memory=True,
    drop_last=False
)

# Define Model

In [15]:
class RSNA24Model(nn.Module):
    def __init__(self, model_name, in_c=30, n_classes=75, pretrained=True, features_only=False):
        super().__init__()
        self.model = timm.create_model(
                                    model_name,
                                    pretrained=pretrained, 
                                    features_only=features_only,
                                    in_chans=in_c,
                                    num_classes=n_classes,
                                    global_pool='avg'
                                    )
    
    def forward(self, x):
        y = self.model(x)
        return y

# Load Models

In [16]:
models = []

In [17]:
import glob
# CKPT_PATHS = glob.glob('/kaggle/input/rsna-model/pytorch/default/1/rsna24-results/best_wll_model_fold-*.pt')
CKPT_PATHS = glob.glob('/kaggle/input/rsna2024-lsdc-training-baseline/rsna24-results/best_wll_model_fold-*.pt')
CKPT_PATHS = sorted(CKPT_PATHS)

In [18]:
for i, cp in enumerate(CKPT_PATHS):
    print(f'loading {cp}...')
    model = RSNA24Model(MODEL_NAME, IN_CHANS, N_CLASSES, pretrained=False)
    if torch.device('cpu'):
        state_dict = torch.load(cp, map_location=torch.device('cpu')) 
        model.load_state_dict(torch.load(cp, map_location=torch.device('cpu')), strict=False)
    else:
        state_dict = torch.load(cp)
        model.load_state_dict(torch.load(cp), strict=False)
    

    model.eval()
    model.half()
    model.to(device)
    models.append(model)

loading /kaggle/input/rsna2024-lsdc-training-baseline/rsna24-results/best_wll_model_fold-0.pt...
loading /kaggle/input/rsna2024-lsdc-training-baseline/rsna24-results/best_wll_model_fold-1.pt...
loading /kaggle/input/rsna2024-lsdc-training-baseline/rsna24-results/best_wll_model_fold-2.pt...
loading /kaggle/input/rsna2024-lsdc-training-baseline/rsna24-results/best_wll_model_fold-3.pt...
loading /kaggle/input/rsna2024-lsdc-training-baseline/rsna24-results/best_wll_model_fold-4.pt...


# Inference loop

In [19]:
if torch.device('cpu'):
    # Ensure the model is in full precision
    for m in models:
        m = m.float()


autocast = torch.cuda.amp.autocast(enabled=USE_AMP, dtype=torch.half)
y_preds = []
row_names = []

with tqdm(test_dl, leave=True) as pbar:
    with torch.no_grad():
        for idx, (x, si) in enumerate(pbar):
            x = x.to(device)
            pred_per_study = np.zeros((25, 3))
            
            for cond in CONDITIONS:
                for level in LEVELS:
                    row_names.append(si[0] + '_' + cond + '_' + level)
            
            with autocast:
                for m in models:
                    y = m(x)[0]
                    for col in range(N_LABELS):
                        pred = y[col*3:col*3+3]
                        y_pred = pred.float().softmax(0).cpu().numpy()
                        pred_per_study[col] += y_pred / len(models)
                y_preds.append(pred_per_study)

y_preds = np.concatenate(y_preds, axis=0)

100%|██████████| 1/1 [00:06<00:00,  6.12s/it]


# Make Submission

In [20]:
sub = pd.DataFrame()
sub['row_id'] = row_names
sub[LABELS] = y_preds
sub.head(25)

,row_id,normal_mild,moderate,severe
0,44036939_spinal_canal_stenosis_l1_l2,0.384177,0.360840,0.254983
1,44036939_spinal_canal_stenosis_l2_l3,0.184678,0.419238,0.396084
2,44036939_spinal_canal_stenosis_l3_l4,0.132314,0.452070,0.415616
3,44036939_spinal_canal_stenosis_l4_l5,0.211898,0.250336,0.537767
4,44036939_spinal_canal_stenosis_l5_s1,0.733873,0.143866,0.122261
5,44036939_left_neural_foraminal_narrowing_l1_l2,0.408383,0.544523,0.047095
6,44036939_left_neural_foraminal_narrowing_l2_l3,0.287050,0.571474,0.141476
7,44036939_left_neural_foraminal_narrowing_l3_l4,0.209938,0.505964,0.284098
8,44036939_left_neural_foraminal_narrowing_l4_l5,0.122861,0.566228,0.310911
9,44036939_left_neural_foraminal_narrowing_l5_s1,0.169073,0.388842,0.442085


In [21]:
sub.to_csv('submission.csv', index=False)
pd.read_csv('submission.csv').head()

,row_id,normal_mild,moderate,severe
0,44036939_spinal_canal_stenosis_l1_l2,0.384177,0.360840,0.254983
1,44036939_spinal_canal_stenosis_l2_l3,0.184678,0.419238,0.396084
2,44036939_spinal_canal_stenosis_l3_l4,0.132314,0.452070,0.415616
3,44036939_spinal_canal_stenosis_l4_l5,0.211898,0.250336,0.537767
4,44036939_spinal_canal_stenosis_l5_s1,0.733873,0.143866,0.122261


# Conclusion
We created the dataset, performed training, and inference in this notebook. 

This competition is a bit complicated to handle the dataset, so there may be a better way.

I think there are many other areas to improve in my notebook. I hope you can learn from my notebook and get a better score.